In [17]:
E=210e6 #kN/m^2
mu=0.3  #poissons ratio
t=0.025 #m
w=3000 #kN/m^2



In [2]:
#STEP1 CREATING MEMBER STIFFNESS MATRICES 
import numpy as np
import sys

np.set_printoptions(precision=3,threshold=sys.maxsize,nanstr=str,sign='+',suppress=True)
"""Basic Functions"""

def LinearTriangleElementStiffness(E,mu,t,xi,yi,xj,yj,xm,ym,p):
    """Creates CST stiffness matrix
        p==1 for plane stress
        p==2 for plane strain"""
    A = (xi*(yj-ym)+ xj*(ym-yi) + xm*(yi-yj))/2
    betai = yj-ym
    betaj = ym-yi
    betam = yi-yj
    gammai = xm-xj
    gammaj = xi-xm
    gammam = xj-xi
    B=np.asarray([[betai, 0 ,betaj, 0, betam, 0],[0, gammai, 0, gammaj, 0 ,gammam],[gammai,betai,gammaj,betaj,gammam,betam]])

    B=B*(1/(2*A))
    if p==1:
        D=(E/(1-mu*mu))*np.asarray([[1,mu,0],[mu,1,0],[0,0,(1-mu)/2]])
    elif p==2:
        D=(E/((1+mu)*(1-2*mu)))*np.asarray([[1-mu,mu,0],[mu,1-mu,0],[0,0,(1-2*mu)/2]])
    
    return t*A*np.dot(B.T,np.dot(D,B))
    

def LinearTriangleAssemble(K :np.ndarray, k, i, j, m):
    """Step Assembly of k(one member stiffness matrix) into K(Global stiffness)"""
    """Note: This function assume indexing i,j,m are starts from 1 not 0 """
    K[2*i-1-1, 2*i-1-1] = K[2*i-1-1, 2*i-1-1] + k[1-1, 1-1]
    K[2*i-1-1, 2*i-1] = K[2*i-1-1,  2*i-1] + k[1-1, 2-1]
    K[2*i-1-1, 2*j-1-1] = K[2*i-1-1,  2*j-1-1] + k[1-1, 3-1]
    K[2*i-1-1,  2*j-1] = K[2*i-1-1,  2*j-1] + k[1-1, 4-1]
    K[2*i-1-1,  2*m-1-1] = K[2*i-1-1,  2*m-1-1] + k[1-1, 5-1]
    K[2*i-1-1,  2*m-1] = K[2*i-1-1,  2*m-1] + k[1-1, 6-1]
    K[2*i-1, 2*i-1-1] = K[2*i-1, 2*i-1-1] + k[2-1, 1-1]
    K[2*i-1, 2*i-1] = K[2*i-1, 2*i-1] + k[2-1, 2-1]
    K[2*i-1, 2*j-1-1] = K[2*i-1, 2*j-1-1] + k[2-1, 3-1]
    K[2*i-1, 2*j-1] = K[2*i-1, 2*j-1] + k[2-1, 4-1]
    K[2*i-1, 2*m-1-1] = K[2*i-1, 2*m-1-1] + k[2-1, 5-1]
    K[2*i-1, 2*m-1] = K[2*i-1, 2*m-1] + k[2-1, 6-1]
    K[2*j-1-1, 2*i-1-1] = K[2*j-1-1,  2*i-1-1] + k[3-1, 1-1]
    K[2*j-1-1,  2*i-1] = K[2*j-1-1,  2*i-1] + k[3-1, 2-1]
    K[2*j-1-1,  2*j-1-1] = K[2*j-1-1,  2*j-1-1] + k[3-1, 3-1]
    K[2*j-1-1,  2*j-1] = K[2*j-1-1,  2*j-1] + k[3-1, 4-1]
    K[2*j-1-1,  2*m-1-1] = K[2*j-1-1,  2*m-1-1] + k[3-1, 5-1]
    K[2*j-1-1,  2*m-1] = K[2*j-1-1, 2*m-1] + k[3-1, 6-1]
    K[2*j-1, 2*i-1-1] = K[2*j-1, 2*i-1-1] + k[4-1, 1-1]
    K[2*j-1, 2*i-1] = K[2*j-1, 2*i-1] + k[4-1, 2-1]
    K[2*j-1, 2*j-1-1] = K[2*j-1, 2*j-1-1] + k[4-1, 3-1]
    K[2*j-1, 2*j-1] = K[2*j-1, 2*j-1] + k[4-1, 4-1]
    K[2*j-1, 2*m-1-1] = K[2*j-1, 2*m-1-1] + k[4-1, 5-1]
    K[2*j-1, 2*m-1] = K[2*j-1, 2*m-1] + k[4-1, 6-1]
    K[2*m-1-1,  2*i-1-1] = K[2*m-1-1,  2*i-1-1] + k[5-1, 1-1]
    K[2*m-1-1,  2*i-1] = K[2*m-1-1, 2*i-1] + k[5-1, 2-1]
    K[2*m-1-1,  2*j-1-1] = K[2*m-1-1,  2*j-1-1] + k[5-1, 3-1]
    K[2*m-1-1,  2*j-1] = K[2*m-1-1,  2*j-1] + k[5-1, 4-1]
    K[2*m-1-1,  2*m-1-1] = K[2*m-1-1,  2*m-1-1] + k[5-1, 5-1]
    K[2*m-1-1, 2*m-1] = K[2*m-1-1,  2*m-1] + k[5-1, 6-1]
    K[2*m-1, 2*i-1-1] = K[2*m-1, 2*i-1-1] + k[6-1, 1-1]
    K[2*m-1, 2*i-1] = K[2*m-1, 2*i-1] + k[6-1, 2-1]
    K[2*m-1, 2*j-1-1] = K[2*m-1, 2*j-1-1] + k[6-1, 3-1]
    K[2*m-1, 2*j-1] = K[2*m-1, 2*j-1] + k[6-1, 4-1]
    K[2*m-1, 2*m-1-1] = K[2*m-1, 2*m-1-1] + k[6-1, 5-1]
    K[2*m-1, 2*m-1] = K[2*m-1, 2*m-1] + k[6-1, 6-1]
    
    return K
    
def LinearTriangleElementStresses(E,mu,xi,yi,xj,yj,xm,ym,p,u):
    A = (xi*(yj-ym) + xj*(ym-yi) + xm*(yi-yj))/2
    betai = yj-ym
    betaj = ym-yi
    betam = yi-yj
    gammai = xm-xj
    gammaj = xi-xm
    gammam = xj-xi
    B = np.asarray([[betai, 0, betaj, 0, betam, 0],[0, gammai, 0, gammaj, 0 ,gammam],[ gammai, betai, gammaj, betaj ,gammam, betam]])/(2*A)
    if p == 1:
        D = (E/(1-mu*mu))*np.asarray([[1, mu, 0],[ mu, 1, 0],[ 0 ,0, (1-mu)/2]])
    elif p == 2:
        D = (E/((1+mu)*(1-2*mu)))*np.asarray([[1-mu, mu, 0],[ mu, 1-mu, 0],[ 0, 0, (1-2*mu)/2]])
  
    return np.dot(D,np.dot(B,u))

def LinearTriangleElementPStresses(sigma):
    R = (sigma[0] + sigma[1])/2
    Q = ((sigma[0] - sigma[1])/2)**2 + sigma[2]*sigma[2]
    M = 2*sigma[2]/(sigma[0] - sigma[1])
    s1 = R + np.sqrt(Q)
    s2 = R - np.sqrt(Q)
    theta = (np.arctan(M)/2)*180/np.pi
    return np.asarray([s1 , s2 , theta])
    

In [32]:
Node_coord=[(0,0),(0.5,0),(0.5,0.25),(0,0.25),(0.125,0.125)]    #5 nodes
Elements_connectivity=[(0,1,4),(1,2,4),(2,3,4),(3,0,4)]         #4 Elem !Order imp: must be counterclockwise

#STEP 1: Creating member stifness matrices
Elem_k=[]
for i in Elements_connectivity:
    xi,yi=Node_coord[i[0]][0],Node_coord[i[0]][1]
    xj,yj=Node_coord[i[1]][0],Node_coord[i[1]][1]
    xm,ym=Node_coord[i[2]][0],Node_coord[i[2]][1]
    Elem_k+=[LinearTriangleElementStiffness(E,mu,t,xi,yi,xj,yj,xm,ym,1)]
print("Element1 stiffness matrices")
print(Elem_k)

Element1 stiffness matrices
[array([[ +2992788.462,  +1406250.   ,    +36057.692,   -540865.385,
         -3028846.154,   -865384.615],
       [ +1406250.   ,  +6742788.462,   -396634.615,  +1911057.692,
         -1009615.385,  -8653846.154],
       [   +36057.692,   -396634.615,   +973557.692,   -468750.   ,
         -1009615.385,   +865384.615],
       [  -540865.385,  +1911057.692,   -468750.   ,   +973557.692,
         +1009615.385,  -2884615.385],
       [ -3028846.154,  -1009615.385,  -1009615.385,  +1009615.385,
         +4038461.538,        +0.   ],
       [  -865384.615,  -8653846.154,   +865384.615,  -2884615.385,
               +0.   , +11538461.538]]), array([[+1995192.308,  -937500.   , -1033653.846,   -72115.385,
         -961538.462, +1009615.385],
       [ -937500.   , +4495192.308,   +72115.385, -4158653.846,
         +865384.615,  -336538.462],
       [-1033653.846,   +72115.385, +1995192.308,  +937500.   ,
         -961538.462, -1009615.385],
       [  -72115.385, -4

In [33]:
#STEP 2 ASSEMBLING
n=2*len(Node_coord)        #Twice the number of nodes
K = np.zeros((n, n))    #Global stiffness matrix 
print(K.shape)


(10, 10)


In [34]:

#Assembling All the elements Linear Triangle
counter=0
for i in Elements_connectivity:
    K=LinearTriangleAssemble(K,Elem_k[counter],i[0]+1,i[1]+1,i[2]+1)
    counter+=1
print("Global Stiffness Matrix(Assembled)")
print(K.shape)
print(K)

Global Stiffness Matrix(Assembled)
(10, 10)
[[ +4939903.846  +2343750.       +36057.692   -540865.385        +0.
         +0.      +937500.       +72115.385  -5913461.538  -1875000.   ]
 [ +2343750.     +8689903.846   -396634.615  +1911057.692        +0.
         +0.       -72115.385   -937500.     -1875000.     -9663461.538]
 [   +36057.692   -396634.615  +2968750.     -1406250.     -1033653.846
     -72115.385        +0.           +0.     -1971153.846  +1875000.   ]
 [  -540865.385  +1911057.692  -1406250.     +5468750.       +72115.385
   -4158653.846        +0.           +0.     +1875000.     -3221153.846]
 [       +0.           +0.     -1033653.846    +72115.385  +2968750.
   +1406250.       +36057.692   +396634.615  -1971153.846  -1875000.   ]
 [       +0.           +0.       -72115.385  -4158653.846  +1406250.
   +5468750.      +540865.385  +1911057.692  -1875000.     -3221153.846]
 [  +937500.       -72115.385        +0.           +0.       +36057.692
    +540865.385  +4939903.

In [35]:
##STEP 3: Forming Disp_Vec, Force_vec and applying Boundary conditions
U=np.zeros((n,1))   #Disp vec
F=np.zeros((n,1))   #Force vec

#Cnditions
F[2,0]=9.375
F[3,0]=0
F[4,0]=9.375
F[5,0]=0
F[8,0]=0
F[9,0]=0

#Partitioning matrices according to known and unknown displacements
Up=U[[2,3,4,5,8,9]]
Fp=F[[2,3,4,5,8,9]]
Kpp=K[[2,3,4,5,8,9]]
Kpp=Kpp[:,[2,3,4,5,8,9]]
#Calculate Up=(Kpp^-1)*Fp
Up=np.dot(np.linalg.inv(Kpp),Fp)
print("Nodal Displacements of [U2x U2y U3x U3y U5x U5y]' (in m)")
print(Up*1e5)




Nodal Displacements of [U2x U2y U3x U3y U5x U5y]' (in m)
[[+0.689]
 [+0.065]
 [+0.689]
 [-0.065]
 [+0.157]
 [-0.   ]]


In [37]:
#STEP 4 Post Processing

#Add back calculated Up
U[[2,3,4,5,8,9]]=Up

#Calculate force
F=np.dot(K,U)
print("Force Vector [F1x0 F1y F2x F2y F3x F3y F4x F4y F5x F5y](in kN)")
print(F)

Force Vector [F1x0 F1y F2x F2y F3x F3y F4x F4y F5x F5y](in kN)
[[-9.375]
 [-4.432]
 [+9.375]
 [-0.   ]
 [+9.375]
 [+0.   ]
 [-9.375]
 [+4.432]
 [+0.   ]
 [-0.   ]]


In [42]:
##STEP 6: element stresses calc

#Element disp vector
#(0,1,4),(1,2,4),(2,3,4),(3,0,4)
u1=np.asarray([U[0],U[1],U[2],U[3],U[8],U[9]])
u2=np.asarray([U[2],U[3],U[4],U[5],U[8],U[9]])
u3=np.asarray([U[4],U[5],U[6],U[7],U[8],U[9]])
u4=np.asarray([U[6],U[7],U[0],U[1],U[8],U[9]])
u=[u1,u2,u3,u4]

In [43]:

Elem_stress_sigma=[]
counter=0
for i in Elements_connectivity:
    xi,yi=Node_coord[i[0]][0],Node_coord[i[0]][1]
    xj,yj=Node_coord[i[1]][0],Node_coord[i[1]][1]
    xm,ym=Node_coord[i[2]][0],Node_coord[i[2]][1]
    Elem_stress_sigma+=[LinearTriangleElementStresses(E,mu,xi,yi,xj,yj,xm,ym,1,u[counter])]
    counter+=1

counter=1
for i in Elem_stress_sigma:
    sigma=i
    print("Element" +str(counter)+ "stress(in Mpa)[sigma_x sigma_y tau_xy]")
    print(sigma)
    counter+=1

Element1stress(in Mpa)[sigma_x sigma_y tau_xy]
[[+3089.912]
 [ +654.24 ]
 [   +5.117]]
Element2stress(in Mpa)[sigma_x sigma_y tau_xy]
[[+2915.205]
 [ -216.374]
 [   -0.   ]]
Element3stress(in Mpa)[sigma_x sigma_y tau_xy]
[[+3089.912]
 [ +654.24 ]
 [   -5.117]]
Element4stress(in Mpa)[sigma_x sigma_y tau_xy]
[[+2894.737]
 [ +868.421]
 [   -0.   ]]


In [4]:
#STEP 8 Principal stresses
counter=1
for i in Elem_stress_sigma:
    sigma=i
    s1=LinearTriangleElementPStresses(sigma)
    print("Element "+str(counter) + " principal stress(in Mpa)[sigma1 sigma2 Theta_p]")
    print(s1)
    counter+=1
    

NameError: name 'Elem_stress_sigma' is not defined